# Decision Tree Classifier on waiting times

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.metrics import classification_report
from tqdm import tqdm
from statistics import mean
import math
from statistics import mean
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score, accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

Let's load all the different datasets with encoding techniques, erase the nulls and delete the rows with 0 waiting time

In [2]:
hp_oHe = pd.read_csv('HP_OHE_3class.csv')
hp_oHe.drop(hp_oHe.tail(17).index,inplace=True) 
hp_oHe = hp_oHe.drop('Unnamed: 0',axis=1)

hp_ME = pd.read_csv("harryPotterClean.csv")
hp_ME.drop(hp_ME.tail(17).index,inplace=True) 
hp_ME = hp_ME.drop('Unnamed: 0',axis=1)

hp_OE = pd.read_csv("harryPotterCleanOE.csv")
hp_OE.drop(hp_OE.tail(17).index,inplace=True) 
hp_OE = hp_OE.drop('Unnamed: 0',axis=1)

Important Functions

In [18]:
def getXandY(df):
    df.drop(df.tail(20).index,inplace=True) 
    x = df.drop(['HP_Forbidden_clean'],axis=1)
    y = df.HP_Forbidden_clean
    return(x,y)

def trainTest(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True)
    return(X_train, X_test, y_train, y_test)

#Function to perform oversampling
def overSampling(X_train, y_train, y_test, method):
    X_train_os, y_train_os= method.fit_resample(X_train, y_train)
    # Check the number of records after over sampling
    #print(sorted(Counter(y_train_os).items())) 
    return(X_train_os, y_train_os)

smote = SMOTE(random_state=42)


def testModel(df,var_order,n_vars,n_loops,method):
    highest = 0
    for j in tqdm(range(1,n_vars)):
        #split our dataframe into X and Y
        x,y=getXandY(df)
        #create the lists to store metrics
        acc = []
        rec = []
        preci = []
        f1 = []
        for i in range(n_loops):
            #split the dataFrame into test and train
            X_train, X_test, y_train, y_test = trainTest(x,y)
            #Oversample the train dataset with SMOTE
            X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
            #define the variables order 
            X_train_os_r = X_train_os[var_order]
            X_test_r = X_test[var_order]
            df1= X_train_os_r.iloc[:, 0:j] #use only part of the variables
            
            #create and train decision trees
            tree_clf = DecisionTreeClassifier()
            tree_clf.fit(df1, y_train_os)
        
            y_pred=tree_clf.predict(X_test_r.iloc[:, 0:j])
            ac=metrics.accuracy_score(y_test, y_pred)
            acc.append(ac)
            p=metrics.precision_score(y_test, y_pred,average='macro')
            preci.append(p)
            r=metrics.recall_score(y_test, y_pred,average='macro')
            rec.append(r)
            f=metrics.f1_score(y_test, y_pred, average='macro')
            f1.append(f)
        print(df1.columns)
        print("For {} features: \n Accuracy: {} \n Precision: {} \n Recall: {} \n F1 score: {}".format(
        j,mean(acc),mean(preci),mean(rec),mean(f1)))
        
        if mean(acc)>highest:
            highest = mean(acc)
            best = "best accuracy = {}, with {} features, with {}".format(mean(acc),j,method)
        print(best)
        #print(classification_report(y_test, y_pred))
    print(best)
        
def analizeDF(df,order,n_vars,n_loops):
    for i in range(len(order)):
        print('------------------------- Analyzing method {} -------------------------'.format(method[i]))
        print('The variable order is: \n {}'.format(order[i]))
        testModel(df,order[i],n_vars,n_loops,method[i])
        print('\n \n')

## One hot encoding

In [4]:
#Variable order in one hot encoding:
pear_corrO = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'fog',	'overcast clouds',	'heavy intensity rain',	'minute',	'haze',	'thunderstorm with light rain',	'scattered clouds',	'clear sky',	'mist',	'light intensity drizzle',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'year',	'light rain',	'humidity',	'hour',	'Pandemic']
kend_corrO = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'shower rain',	'broken clouds',	'heavy intensity rain',	'fog',	'year',	'overcast clouds',	'minute',	'haze',	'thunderstorm with light rain',	'mist',	'light intensity drizzle',	'scattered clouds',	'clear sky',	'few clouds',	'thunderstorm',	'very heavy rain',	'moderate rain',	'thunderstorm with rain',	'light rain',	'humidity',	'hour',	'Pandemic']
mutInf_classO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_regO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'moderate rain',	'mist',	'overcast clouds',	'clear sky',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'very heavy rain',	'fog',	'haze',	'thunderstorm with light rain',	'light intensity drizzle']
mutInf_class2O = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek',	'year',	'holiday',	'shower rain',	'light rain',	'thunderstorm',	'fog',	'broken clouds',	'Pandemic',	'thunderstorm with rain',	'light intensity drizzle',	'thunderstorm with light rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds',	'clear sky']
varThreO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',]
mrmrO = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'heavy intensity rain',	'light rain',	'broken clouds',	'scattered clouds',	'thunderstorm with rain',	'few clouds',	'thunderstorm',	'shower rain',	'heavy intensity rain',	'mist',	'scattered clouds',	'very heavy rain',	'overcast clouds',	'moderate rain',	'minute',	'haze',	'few clouds']

orderOHE=[pear_corrO,kend_corrO,mutInf_classO,mutInf_regO,mutInf_class2O,varThreO,mrmrO]

method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'varianceThreshold','MRMR']

In [19]:
analizeDF(hp_oHe,orderOHE,28,n_loops=20)

  0%|          | 0/27 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'shower rain', 'broken clouds', 'fog', 'overcast clouds', 'heavy intensity rain', 'minute', 'haze', 'thunderstorm with light rain', 'scattered clouds', 'clear sky', 'mist', 'light intensity drizzle', 'few clouds', 'thunderstorm', 'very heavy rain', 'moderate rain', 'thunderstorm with rain', 'year', 'light rain', 'humidity', 'hour', 'Pandemic']


  4%|▎         | 1/27 [00:03<01:25,  3.28s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.47597692105953315 
 Precision: 0.3639868984750958 
 Recall: 0.3577108616481101 
 F1 score: 0.35190888457802116
best accuracy = 0.47597692105953315, with 1 features, with Pearson_correlation


  7%|▋         | 2/27 [00:06<01:20,  3.20s/it]

...

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute'],
      dtype='object')
For 25 features: 
 Accuracy: 0.8501853911753801 
 Precision: 0.8135063836808353 
 Recall: 0.8153063075773118 
 F1 score: 0.8142406178059793
best accuracy = 0.8597357944263482, with 14 features, with MRMR


 96%|█████████▋| 26/27 [01:04<00:02,  2.83s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze'],
      dtype='object')
For 26 features: 
 Accuracy: 0.8531958379784467 
 Precision: 0.8170229555905253 
 Recall: 0.8189211034652957 
 F1 score: 0.8177384891842577
best accuracy = 0.8597357944263482, with 14 features, with MRMR


100%|██████████| 27/27 [01:07<00:00,  2.50s/it]

Index(['month', 'day', 'year', 'hour', 'minute', 'holiday', 'dayOfTheWeek',
       'temperature', 'humidity', 'pressure', 'heavy intensity rain',
       'light rain', 'broken clouds', 'scattered clouds',
       'thunderstorm with rain', 'few clouds', 'thunderstorm', 'shower rain',
       'heavy intensity rain', 'mist', 'scattered clouds', 'very heavy rain',
       'overcast clouds', 'moderate rain', 'minute', 'haze', 'few clouds'],
      dtype='object')
For 27 features: 
 Accuracy: 0.8494785847299814 
 Precision: 0.8102217887582863 
 Recall: 0.8155271898333033 
 F1 score: 0.8126244936372491
best accuracy = 0.8597357944263482, with 14 features, with MRMR
best accuracy = 0.8597357944263482, with 14 features, with MRMR

 



## Manual Encoding

In [21]:
#Replace times ending in 5 and also compacting the rest to have only 6 classes
b=hp_ME.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df3=pd.DataFrame(b)
df3.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis3=pd.concat([hp_ME, df3], axis=1)
hp3 = hp_bis3.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp3.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)
hp3 = hp3[hp3.HP_Forbidden_clean != 0] #delete rows with 0 min
hp3.HP_Forbidden_clean.unique()

array([2., 1., 3.])

In [24]:
HP_ME = hp3

pear_corr = ['temperature','holiday','day','month','pressure','dayOfTheWeek',
            'report','minute','year','humidity','hour',	'Pandemic']
kend_corr = ['temperature','holiday','month','day','pressure','dayOfTheWeek','report','year',
             'minute','humidity','hour','Pandemic']
mutInf_class = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_reg = ['month','day','year','hour','minute','holiday','dayOfTheWeek','Pandemic',
                'temperature','humidity','pressure','report']
mutInf_class2 = ['day','temperature','month','humidity','hour','dayOfTheWeek','pressure',
                 'holiday','year','report','minute','Pandemic']
varThre = ['month','day','year','hour','minute','holiday','dayOfTheWeek',
           'temperature','humidity','pressure','report']
mrmr = ['temperature','dayOfTheWeek','hour','holiday','humidity','day',
        'year','month','pressure','minute','report']
method = ['Pearson_correlation','Kendalls_correlation','mutualInformation_classification',
          'mutualInformation_reggression','mutualInformation_classification2',
          'variableThreshold','MRMR']

orderME=[pear_corr,kend_corr,mutInf_class,mutInf_reg,mutInf_class2,varThre,mrmr]

In [25]:
analizeDF(HP_ME,orderME,13,n_loops=20)

  0%|          | 0/12 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  8%|▊         | 1/12 [00:01<00:21,  1.98s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.4634708994708995 
 Precision: 0.3700910940418719 
 Recall: 0.36331642781462353 
 F1 score: 0.3559762610948642
best accuracy = 0.4634708994708995, with 1 features, with Pearson_correlation


 17%|█▋        | 2/12 [00:03<00:19,  1.93s/it]

...

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.851450813871196 
 Precision: 0.8276560042166824 
 Recall: 0.82959301412462 
 F1 score: 0.8285145783598504
best accuracy = 0.8638162544169612, with 9 features, with MRMR


 92%|█████████▏| 11/12 [00:22<00:02,  2.26s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.8520316560359084 
 Precision: 0.8278368306556021 
 Recall: 0.8295627377191694 
 F1 score: 0.8286079707049163
best accuracy = 0.8638162544169612, with 9 features, with MRMR


100%|██████████| 12/12 [00:24<00:00,  2.06s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 12 features: 
 Accuracy: 0.8484977525431748 
 Precision: 0.8232184423933686 
 Recall: 0.8264645071398894 
 F1 score: 0.8247185059332616
best accuracy = 0.8638162544169612, with 9 features, with MRMR
best accuracy = 0.8638162544169612, with 9 features, with MRMR

 



## Ordinal Encoding

In [26]:
hp_OE.rename(columns = {'day.1':'dayOfTheWeek'}, inplace = True)

#Replace times ending in 5 and also compacting the rest to have only 6 classes
a=hp_OE.Harry_Potter_and_the_Forbidden.replace([
5, 10, 11, 15, 20, 25, 30, 35, 40, 45,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,180.0], 
[1,1,  1,  1,  1,  1,  1,  2,  2,  2, 2,   2,   2,   3,   3,   3,   3,   3,   3,   3,   3,    3,    3,    3,     3,    3,   3,    3,     3,    3,    3])

df1=pd.DataFrame(a)
df1.rename(columns = {'Harry_Potter_and_the_Forbidden':'HP_Forbidden_clean'}, inplace = True)
hp_bis=pd.concat([hp_OE, df1], axis=1)
hp2 = hp_bis.drop('Harry_Potter_and_the_Forbidden',axis=1)
hp2 = hp2[hp2.HP_Forbidden_clean != 0] #delete rows with 0 min
hp2.HP_Forbidden_clean.unique()

#Variable order in ordinal encoding:
pear_corrOE = ['temperature',	'holiday',	'day',	'month',	'pressure',	'dayOfTheWeek',	'report',	'minute',	'year',	'humidity',	'hour',	'Pandemic']
kend_corrOE = ['temperature',	'holiday',	'month',	'day',	'pressure',	'dayOfTheWeek',	'report',	'year',	'minute',	'humidity',	'hour',	'Pandemic',]
mutInf_classOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_regOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'Pandemic',	'temperature',	'humidity',	'pressure',	'report',]
mutInf_class2OE = ['day',	'temperature',	'month',	'humidity',	'hour',	'dayOfTheWeek',	'pressure',	'holiday',	'year',	'report',	'minute',	'Pandemic',]
varThreOE = ['month',	'day',	'year',	'hour',	'minute',	'holiday',	'dayOfTheWeek',	'temperature',	'humidity',	'pressure',	'report']
mrmrOE = ['temperature',	'dayOfTheWeek',	'hour',	'holiday',	'humidity',	'day',	'year',	'month',	'pressure',	'minute',	'report']

orderOE=[pear_corrOE,kend_corrOE,mutInf_classOE,mutInf_regOE,mutInf_class2OE,varThreOE,mrmrOE]

In [27]:
analizeDF(hp2,orderOE,13,n_loops=20)

  0%|          | 0/12 [00:00<?, ?it/s]

------------------------- Analyzing method Pearson_correlation -------------------------
The variable order is: 
 ['temperature', 'holiday', 'day', 'month', 'pressure', 'dayOfTheWeek', 'report', 'minute', 'year', 'humidity', 'hour', 'Pandemic']


  8%|▊         | 1/12 [00:02<00:22,  2.01s/it]

Index(['temperature'], dtype='object')
For 1 features: 
 Accuracy: 0.462010582010582 
 Precision: 0.3680411833372365 
 Recall: 0.3616123656891221 
 F1 score: 0.35424425515397745
best accuracy = 0.462010582010582, with 1 features, with Pearson_correlation


 17%|█▋        | 2/12 [00:03<00:19,  1.97s/it]

...

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute'],
      dtype='object')
For 10 features: 
 Accuracy: 0.8503538570417551 
 Precision: 0.8252442238796531 
 Recall: 0.8283100168246242 
 F1 score: 0.8266474008389586
best accuracy = 0.8682096584216725, with 9 features, with MRMR


 92%|█████████▏| 11/12 [00:22<00:02,  2.24s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 11 features: 
 Accuracy: 0.8506850933144342 
 Precision: 0.8262711629134808 
 Recall: 0.8275451348023408 
 F1 score: 0.8268235678550325
best accuracy = 0.8682096584216725, with 9 features, with MRMR


100%|██████████| 12/12 [00:24<00:00,  2.04s/it]

Index(['temperature', 'dayOfTheWeek', 'hour', 'holiday', 'humidity', 'day',
       'year', 'month', 'pressure', 'minute', 'report'],
      dtype='object')
For 12 features: 
 Accuracy: 0.8511237284125858 
 Precision: 0.8266188282620687 
 Recall: 0.8293046372190976 
 F1 score: 0.8278628793668295
best accuracy = 0.8682096584216725, with 9 features, with MRMR
best accuracy = 0.8682096584216725, with 9 features, with MRMR

 



## Hyper parameter tuning of Dicision Trees

In [63]:
crit=['gini', 'entropy']
split=['best', 'random']
max_d=[1,2,5,10,20,50,100,250,500,1000,2000,5000,10000]
min_samp_split=[2,5,10,20,50,100]

In [58]:
def tuneDecisionT(X,Y,mu,crit,split,max_d,min_samp_split,loops):
    arr = []
    highest=0
    for i in range(len(crit)):
        for k in range(len(split)):
            for a in range(len(max_d)):
                for b in range(len(min_samp_split)):
                    for l in range(loops):
                        X_train, X_test, y_train, y_test = trainTest(X,Y)
                        #Oversample the train dataset with SMOTE
                        X_train_os, y_train_os=overSampling(X_train, y_train, y_test, smote)
                        #define the variables order 
                        X_train_os_r = X_train_os[mu]
                        X_test_r = X_test[mu]
                        tree_clf = DecisionTreeClassifier(criterion=crit[i],
                                                         splitter=split[k],
                                                         max_depth=max_d[a],
                                                         min_samples_split=min_samp_split[b])
                        tree_clf.fit(X_train_os_r,y_train_os)
                        y_pred=tree_clf.predict(X_test_r)
                        score = accuracy_score(y_test, tree_clf.predict(X_test_r))
                        arr.append(score)
                    print("For the parameters crit:{}, split:{}, "
                          "maxDep:{}, maxSamp:{} accuracy is:".format(crit[i],
                                                                     split[k],
                                                                     max_d[a],
                                                                     min_samp_split[b]))
                    mean_acc=mean(arr)
                    print(mean_acc)
                    #check the best configuration
                    if mean_acc > highest:
                        highest = mean_acc
                        description1 = "-----------------Best values = crit:{}, split:{}, ".format(crit[i],
                                                                                 split[k])
                        description2= " maxDep:{}, min_Samp:{} with accuracy: {}---------".format(
                                                                                 max_d[a],
                                                                                 min_samp_split[b],
                                                                                 mean_acc)
                    else:
                        highest = highest
                    arr = []
        print(description1+description2)

In [65]:
X,Y=getXandY(hp_oHe)
mu = ['day',	'temperature',	'month',	'humidity',	'hour',	'pressure','dayOfTheWeek']
loops=10
tuneDecisionT(X,Y,mu,crit,split,max_d,min_samp_split,loops)

For the parameters crit:gini, split:best, maxDep:1, maxSamp:2 accuracy is:
0.3408614668218859
For the parameters crit:gini, split:best, maxDep:1, maxSamp:5 accuracy is:
0.3407838571982926
For the parameters crit:gini, split:best, maxDep:1, maxSamp:10 accuracy is:
0.3414047341870392
For the parameters crit:gini, split:best, maxDep:1, maxSamp:20 accuracy is:
0.3477299185098952
For the parameters crit:gini, split:best, maxDep:1, maxSamp:50 accuracy is:
0.34229724485836244
For the parameters crit:gini, split:best, maxDep:1, maxSamp:100 accuracy is:
0.33554520760574313
For the parameters crit:gini, split:best, maxDep:2, maxSamp:2 accuracy is:
0.4991850989522701
For the parameters crit:gini, split:best, maxDep:2, maxSamp:5 accuracy is:
0.47411719053162593
For the parameters crit:gini, split:best, maxDep:2, maxSamp:10 accuracy is:
0.4449359720605355
For the parameters crit:gini, split:best, maxDep:2, maxSamp:20 accuracy is:
0.4647264260768335
For the parameters crit:gini, split:best, maxDep:2

For the parameters crit:gini, split:random, maxDep:2, maxSamp:5 accuracy is:
0.4426852929763291
For the parameters crit:gini, split:random, maxDep:2, maxSamp:10 accuracy is:
0.44253007372914244
For the parameters crit:gini, split:random, maxDep:2, maxSamp:20 accuracy is:
0.4674427629025999
For the parameters crit:gini, split:random, maxDep:2, maxSamp:50 accuracy is:
0.44447031431897555
For the parameters crit:gini, split:random, maxDep:2, maxSamp:100 accuracy is:
0.42599922390376405
For the parameters crit:gini, split:random, maxDep:5, maxSamp:2 accuracy is:
0.5706247574699262
For the parameters crit:gini, split:random, maxDep:5, maxSamp:5 accuracy is:
0.5574699262708576
For the parameters crit:gini, split:random, maxDep:5, maxSamp:10 accuracy is:
0.5682188591385332
For the parameters crit:gini, split:random, maxDep:5, maxSamp:20 accuracy is:
0.5620488940628638
For the parameters crit:gini, split:random, maxDep:5, maxSamp:50 accuracy is:
0.5580519984478075
For the parameters crit:gini,

For the parameters crit:entropy, split:best, maxDep:2, maxSamp:100 accuracy is:
0.41571594877764845
For the parameters crit:entropy, split:best, maxDep:5, maxSamp:2 accuracy is:
0.6249126891734575
For the parameters crit:entropy, split:best, maxDep:5, maxSamp:5 accuracy is:
0.6259992239037641
For the parameters crit:entropy, split:best, maxDep:5, maxSamp:10 accuracy is:
0.6308886301901435
For the parameters crit:entropy, split:best, maxDep:5, maxSamp:20 accuracy is:
0.623476911136981
For the parameters crit:entropy, split:best, maxDep:5, maxSamp:50 accuracy is:
0.6240977881257276
For the parameters crit:entropy, split:best, maxDep:5, maxSamp:100 accuracy is:
0.6189367481567715
For the parameters crit:entropy, split:best, maxDep:10, maxSamp:2 accuracy is:
0.7868063639891346
For the parameters crit:entropy, split:best, maxDep:10, maxSamp:5 accuracy is:
0.7889406286379511
For the parameters crit:entropy, split:best, maxDep:10, maxSamp:10 accuracy is:
0.7786573535118355
For the parameters 

For the parameters crit:entropy, split:random, maxDep:5, maxSamp:50 accuracy is:
0.5610011641443539
For the parameters crit:entropy, split:random, maxDep:5, maxSamp:100 accuracy is:
0.5570042685292976
For the parameters crit:entropy, split:random, maxDep:10, maxSamp:2 accuracy is:
0.7352735739231665
For the parameters crit:entropy, split:random, maxDep:10, maxSamp:5 accuracy is:
0.740046565774156
For the parameters crit:entropy, split:random, maxDep:10, maxSamp:10 accuracy is:
0.7387272021730694
For the parameters crit:entropy, split:random, maxDep:10, maxSamp:20 accuracy is:
0.7324020178502134
For the parameters crit:entropy, split:random, maxDep:10, maxSamp:50 accuracy is:
0.7166084594489717
For the parameters crit:entropy, split:random, maxDep:10, maxSamp:100 accuracy is:
0.7074117190531626
For the parameters crit:entropy, split:random, maxDep:20, maxSamp:2 accuracy is:
0.8714396585176561
For the parameters crit:entropy, split:random, maxDep:20, maxSamp:5 accuracy is:
0.858634070624